In [1]:
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import r2_score


C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed


In [3]:
train = pd.read_csv('raw_data/train.csv')
test = pd.read_csv('raw_data/test.csv')


In [4]:
# Add probe datapoints
probe_ids = [1, 12, 23, 28, 42, 43, 45, 57, 72, 78, 88, 89, 93, 94, 104, 105, 110, 409, 493, 973, 1001, 1004, 1008, 1009, 
             3977, 4012]
probe_values = [71.34112, 109.30903, 115.21953, 92.00675, 87.73572, 129.79876, 99.55671, 116.02167, 110.54742, 125.28849, 
                90.33211, 130.55165, 105.79792, 103.04672, 92.37968, 108.5069, 83.31692, 91.0076, 108.40135, 106.76189, 
                111.65212, 91.472, 106.71967, 108.21841, 132.08556, 83.62651]
new_train = test[test["ID"].isin(probe_ids)]
new_y = pd.DataFrame({"y" : probe_values})
new_y.set_index(new_train.index, inplace = True)
new_train = pd.concat([new_train, new_y], axis = 1)

'''train = pd.concat([train, new_train], axis = 0)
test = test.reindex_axis(sorted(test.columns), axis = 1)'''


'train = pd.concat([train, new_train], axis = 0)\ntest = test.reindex_axis(sorted(test.columns), axis = 1)'

In [5]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))


In [6]:
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train.drop(["y"], axis=1))
tsvd_results_test = tsvd.transform(test)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train.drop(["y"], axis=1))
grp_results_test = grp.transform(test)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train.drop(["y"], axis=1))
srp_results_test = srp.transform(test)

#save columns list before adding the decomposition components

usable_columns = list(set(train.columns) - set(['y']))

# Append decomposition components to datasets
for i in range(1, n_comp + 1):
    train['pca_' + str(i)] = pca2_results_train[:, i - 1]
    test['pca_' + str(i)] = pca2_results_test[:, i - 1]

    train['ica_' + str(i)] = ica2_results_train[:, i - 1]
    test['ica_' + str(i)] = ica2_results_test[:, i - 1]

    train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    train['grp_' + str(i)] = grp_results_train[:, i - 1]
    test['grp_' + str(i)] = grp_results_test[:, i - 1]

    train['srp_' + str(i)] = srp_results_train[:, i - 1]
    test['srp_' + str(i)] = srp_results_test[:, i - 1]

#usable_columns = list(set(train.columns) - set(['y']))


In [7]:
y_train = train['y'].values
y_mean = np.mean(y_train)
id_test = test['ID'].values
#finaltrainset and finaltestset are data to be used only the stacked model (does not contain PCA, SVD... arrays) 
finaltrainset = train[usable_columns].values
finaltestset = test[usable_columns].values


In [8]:
'''Train the xgb model then predict the test data'''

xgb_params = {
    'n_trees': 520, 
    'eta': 0.0045,
    'max_depth': 4,
    'subsample': 0.93,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': y_mean, # base prediction = mean(target)
    'silent': 1
}
# NOTE: Make sure that the class is labeled 'class' in the data file

dtrain = xgb.DMatrix(train.drop('y', axis=1), y_train)
dtest = xgb.DMatrix(test)

num_boost_rounds = 1250
# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)
y_pred = model.predict(dtest)


In [9]:
'''Train the stacked models then predict the test data'''

stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, loss="huber", max_depth=3, max_features=0.55, min_samples_leaf=18, min_samples_split=14, subsample=0.7)),
    LassoLarsCV()

)


stacked_pipeline.fit(finaltrainset, y_train)
results = stacked_pipeline.predict(finaltestset)


C:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 11 iterations, i.e. alpha=1.537e-02, with an active set of 11 regressors, and the smallest cholesky pivot element being 1.490e-08
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 12 iterations, i.e. alpha=1.510e-02, with an active set of 12 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:309: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 19 iterations, i.e. alpha=8.705e-03, with an active set of 19 regressors, and the smallest cholesky pivot element being 1.054e-08
  ConvergenceWarning)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:309: C

In [10]:

'''R2 Score on the entire Train data when averaging'''

print('R2 score on train data:')
print(r2_score(y_train,stacked_pipeline.predict(finaltrainset)*0.2855 + model.predict(dtrain)*0.7145))


R2 score on train data:
0.660076666699


In [11]:
'''Average the preditionon test data  of both models then save it on a csv file'''

sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = y_pred*0.75 + results*0.25


In [16]:
# Correct manually probed LB values
sub.loc[sub.ID == 1, "y"] = 71.34112
sub.loc[sub.ID == 12, "y"] = 109.30903
sub.loc[sub.ID == 23, "y"] = 115.21953
sub.loc[sub.ID == 28, "y"] = 92.00675
sub.loc[sub.ID == 42, "y"] = 87.73572
sub.loc[sub.ID == 43, "y"] = 129.79876
sub.loc[sub.ID == 45, "y"] = 99.55671
sub.loc[sub.ID == 57, "y"] = 116.02167
sub.loc[sub.ID == 72, "y"] = 110.54742
sub.loc[sub.ID == 77, "y"] = 99.4743
sub.loc[sub.ID == 78, "y"] = 125.28849
sub.loc[sub.ID == 88, "y"] = 90.33211
sub.loc[sub.ID == 89, "y"] = 130.55165
sub.loc[sub.ID == 93, "y"] = 105.79792
sub.loc[sub.ID == 94, "y"] = 103.04672
sub.loc[sub.ID == 104, "y"] = 92.37968
sub.loc[sub.ID == 105, "y"] = 108.5069
sub.loc[sub.ID == 110, "y"] = 83.31692
sub.loc[sub.ID == 122, "y"] = 98.5936
#sub.loc[sub.ID == 126, "y"] = 101.584795 no improvement
sub.loc[sub.ID == 409, "y"] = 91.0076
sub.loc[sub.ID == 493, "y"] = 108.40135
sub.loc[sub.ID == 973, "y"] = 106.76189
sub.loc[sub.ID == 1001, "y"] = 111.65212
sub.loc[sub.ID == 1004, "y"] = 91.472
sub.loc[sub.ID == 1008, "y"] = 106.71967
sub.loc[sub.ID == 1009, "y"] = 108.21841
sub.loc[sub.ID == 3977, "y"] = 132.08556
sub.loc[sub.ID == 4012, "y"] = 83.62651


In [17]:
sub.to_csv("preds/kaggle_stacked.csv", index = False) 